In [1]:
import urllib3
import os
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import time
from lxml import etree
import re
import math
import csv
import sys

In [2]:
requests.adapters.DEFAULT_RETRIES =20
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# 禁用不安全请求警告
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


In [3]:
lsdir = os.listdir('.')
if 'html_doc' not in lsdir:
    os.mkdir('./html_doc')
if 'dataset' not in lsdir:
    os.mkdir('./dataset')
os.chmod('./html_doc',0o777)
aim = 'hug_dataset'
# 下载好的网页文件
html_path = os.path.join(os.getcwd(),'html_doc')
# 解析后的数据文件
web_path = os.path.join(os.getcwd(),'dataset')
# 数据简介
viewer_path = os.path.join(os.getcwd(),'viewer')

In [4]:
# 路由仓库
URL_HUB = dict()
# 分类后的仓库
URL_HUB_class = dict()
# 数据仓库
URL_DATA_HUB = dict()
# 文档仓库
HTML_DOC = dict()
URL_HUB['hug'] = r'https://huggingface.co/'
URL_HUB['hug_dataset'] = r'https://huggingface.co/datasets'

In [5]:
headers = {
        'User-Agent':r'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }
proxies = {
    "http": "http://127.0.0.1:7890",
    "https": "http://127.0.0.1:7890",
}
if os.path.exists(r'Hugging _dataset.html'):
    with open(r'Hugging _dataset.html','r',encoding='utf-8') as f:
        dataset_html = f.read()
else:
    dataset_web_respond = requests.get(url=URL_HUB['hug_dataset'],verify=False)
    dataset_html = dataset_web_respond.text
    with open(r'Hugging _dataset.html','w',encoding='utf-8') as f:
        f.write(dataset_html)
soup = bs(dataset_html,'html.parser')

In [6]:
tags = soup.select('div.mb-20')[0].contents
btags = []
for i in tags:
    try:
        if 'mb-3' in i.get('class'):
            btags.append(i)
    except Exception:
        pass
bdic = dict()
for btag in btags:
    blabel = btag.div.string.rstrip('\t').rstrip('\n')
    sdic = dict()
    for a in btag.find_all('a'):
        url = a.get('href')
        slabel = a.span.string
        sdic[slabel] = 'https://huggingface.co' + url
        print(slabel,sdic[slabel])
    bdic[blabel] = sdic

In [7]:
for k,v in bdic.items():
    print(k)
    for i,(kk,vv) in enumerate(v.items()):
        print(i,kk,vv)

In [8]:
# 小类数据页处理
def find_url(docdir):
    """
    找到path中所有url
    """
    ans = dict()
    with open(docdir,'br') as f:
        dataset_web_respond = f.read()
    soup = bs(dataset_web_respond,'lxml')
    h4_labels = soup.find_all('h4')

    for h4_lable in h4_labels:
        h4_lable = h4_lable.string
        ans[h4_lable] = URL_HUB['hug_dataset'] + '/' + str(h4_lable)
    return ans

def find_cnt(doc):
    """
    获取小类数据总数
    """
    soup = bs(doc)
    # num = soup.select('.ml-3.w-16.font-normal.text-gray-400').string
    tim = soup.find('div',attrs={
        'class':"ml-3 w-16 font-normal text-gray-400"
    }).string.replace(',','')
    print(tim)
    return math.ceil(int(tim)/30)

def download_web(url,path):
    """
    web路径,本地路径
    下载url网页到指定文件夹下
    """
    if not os.path.exists(path):
        try:
            respond = requests.get(url,verify=False,headers={'Connection':'close'}).text
            with open(path,'bw') as f:
                f.write(respond.encode('utf-8'))
        except Exception as e:
            flag = 0
            for i in range(300):
                try:
                    respond = requests.get(url,verify=False,headers={'Connection':'close'}).text
                    with open(path,'bw') as f:
                        f.write(respond.encode('utf-8'))
                except Exception:
                    if i != 299:
                        time.sleep(0.01)
                    else:
                        print('ERROR:',e)
                        print('ERROR',os.path.join(html_path,k+'.html'))
                        return False
                else:
                    # cnt += 1
                    # HTML_DOC[k_] = respond
                    break
        else:
            # 下载成功
            # cnt += 1
            # HTML_DOC[k_] = respond
            pass
    else:
        # 文件已存在
        # with open(os.path.join(path),'r', encoding='utf-8') as f:
        #     HTML_DOC[k_] = f.read()
        #     cnt+=1
        pass
    return True


def str2num(s):
    if isinstance(s,int) or isinstance(s,float):
        return s
    if len(s) == 0:
        return 0
    k = s 
    try:
        s = eval(s.replace(',','').replace(r'\n','').replace(r'\t','').replace(' ','').replace(r'\r',''))
    except Exception as e:
        print('ERRORs2n:',k,e)
        if isinstance(s,int) or isinstance(s,float):
            return s            
        return 0
    return s

def download_web_page(url,path):
    download_web(url,path[:-5]+'0'+path[-5:])
    with open(path[:-5]+'0'+path[-5:], 'rb') as f:
        doc_page = f.read()
    tim = find_cnt(doc_page)
    for i in range(1,tim):
        download_web(url+'&p='+str(i),path[:-5]+str(i)+path[-5:])
        # https://huggingface.co/datasets?task_categories=task_categories:depth-estimation&p=12&sort=trending

In [9]:
# 搜索数据细节函数
def find_page_link(html_doc):
    """
    页面链接
    """
    soup = bs(html_doc,'lxml')
    label = soup.select('.max-w-full>a')[0]
    url = 'https://huggingface.co' + label.get('href')
    # print(url)
    return url

def find_dataset_numb(html_doc):
    index_1 = html_doc.rfind('Number of rows')
    index_2 = html_doc[index_1:].find('<!-- HTML_TAG_START -->') + len('<!-- HTML_TAG_START -->')
    index_3 = html_doc[index_1:].find('<!-- HTML_TAG_END -->')
    ret = html_doc[index_1:][index_2:index_3]
    return str2num(ret)

def find_dataset_size(html_doc:str):
    soup = bs(html_doc,'lxml')
    labels = [x.text for x in soup.select('div.truncate.text-sm')]
    re_num = -1
 
    for label in labels:
        if 'B' in label:
            re = label 
            re_num = str2num(re[0:re.find('B')-2])
            if 'K' in re:
                re_num = re_num / 1024
            elif 'M' in re:
                re_num = re_num
            elif 'G' in re:
                re_num = re_num * 1024
            elif 'T' in re:
                re_num = re_num * 1024 * 1024
            else:
                re_num = re_num / (1024 * 1024)
            return re_num
    return -1

def find_download_time(html_doc):
    dom = etree.HTML(html_doc)
    download_time = -1
    try:
        download_time = dom.xpath('/html/body/div/main/div[2]/section[2]/dl/dd/text()')[0]
    except Exception as e:
        pass
    return str2num(download_time)


def find_dataset_viewer(file_name,html_doc):
    soup = bs(html_doc,'lxml')
    dom = etree.HTML(html_doc)
    try:
        try:
            dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[1]/div[5]/div')[0]
        except Exception:
            dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[1]/div[4]')[0]
        text = etree.tostring(dataset_numb)
        soup_ = bs(text,'lxml').html.body
        p = soup_
        writeinner = soup_.get_text()
        text = re.sub(r'[\r\n]+', '\n', writeinner)
        with open(os.path.join(viewer_path,file_name+'.html'),'bw') as f:
            f.write(text.encode('utf-8'))
    except Exception:
        pass
    finally:
        return(os.path.join(viewer_path,file_name+'.html'))
    

def find_dataset_viewer2(html_doc):
    soup = bs(html_doc,'lxml')
    dom = etree.HTML(html_doc)
    try:
        try:
            dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[1]/div[5]/div')[0]
        except Exception:
            dataset_numb = dom.xpath('/html/body/div/main/div[2]/section[1]/div[4]')[0]
        text = etree.tostring(dataset_numb)
        soup_ = bs(text,'lxml').html.body
        p = soup_
        writeinner = soup_.get_text()
        text = re.sub(r'[\r\n]+', '\n', writeinner)
    except Exception:
        return -1
    else:
        return text 
    
def aio(html_doc):
    page_link = find_page_link(html_doc)
    download_time = find_download_time(html_doc)
    dataset_numb = find_dataset_numb(html_doc)
    dataset_size = find_dataset_size(html_doc)
    dataset_viewer = find_dataset_viewer2(html_doc)
    ans = {
        'page_link':page_link,
        'download_time':download_time,
        'dataset_numb':dataset_numb,
        'dataset_size':dataset_size,
        'dataset_viewer':dataset_viewer
    }
    return ans

In [10]:
# 下载所有小类网页文件
down_error = []
for blabel,aas in bdic.items():
    # 大类文件
    blabel=blabel.replace(' ','_').replace('/','_').replace('\\','_')
    blabel = os.path.join(html_path,blabel)
    if os.path.exists(blabel) == False:
        os.mkdir(blabel)
    print(blabel)
    for i,(slabel,url) in enumerate(aas.items()):
        # 小类文件
        slabel=slabel.replace(' ','_').replace('/','_').replace('\\','_')
        pw_path = os.path.join(blabel,slabel)
        if os.path.exists(pw_path) == False:
            os.mkdir(pw_path)
        print(i,slabel,url)
        try:
            if not download_web_page(url,os.path.join(pw_path,slabel+'.html')):
                down_error.append([url,pw_path])
        except Exception as e:

            print('!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(url)
            print(e)
            break
        else:
            pass
for i in range(len(down_error)):
    url,pw_path = down_error[i]
    while not download_web(url,os.path.join(pw_path,slabel+'.html')):
        pass

In [ ]:
# 下载所有数据
down_error = []

# 查询网页中的数据
bdirs = os.listdir(html_path)
for bdir in tqdm(bdirs[3:]):
    # 打开大类文件夹
    bdir = os.path.join(html_path,bdir)
    print(bdir) 
    sdirs = os.listdir(bdir)
    # print(sdirs)
    for sdir in sdirs:
        # 逐个搜索小类文件夹,sdirs为小类集合
        sdocs = os.listdir(os.path.join(bdir,sdir))
        print(' ',sdir)
        for sdoc in sdocs:
            if '.html' not in sdoc:
                "跳过doc文件"
                continue
            sdoc = os.path.join(bdir,sdir,sdoc)

            # 找到小类网页中的所有数据库
            url_doc = find_url(sdoc)

            doc_path = os.path.join(bdir,sdir,'doc')
            if not os.path.exists(doc_path):
                os.mkdir(doc_path)
            for name,url in url_doc.items():
                name = name.replace(' ','_').replace('/','_').replace('\\','_')
                # print(name,url)
                doc_path_i = os.path.join(doc_path,name+'.html')
                if not os.path.exists(doc_path_i):
                    if not download_web(url,doc_path_i):
                        down_error.append([url,doc_path_i])

In [ ]:
def find_dataset_numb(html_doc):
    index_1 = html_doc.rfind('Number of rows')
    index_2 = html_doc[index_1:].find('<!-- HTML_TAG_START -->') + len('<!-- HTML_TAG_START -->')
    index_3 = html_doc[index_1:].find('<!-- HTML_TAG_END -->')
    ret = html_doc[index_1:][index_2:index_3]
 
    return str2num(ret)

In [ ]:
p = r'D:\桌面\lab_2\lab_2\html_doc\Audio\Audio-to-Audio\doc\allenai_dolma.html'
with open(p,'r') as f:
    text = f.read()
    html_doc = str(text)
    page_link = find_page_link(html_doc)
    download_time = find_download_time(html_doc)
    dataset_numb = find_dataset_numb(html_doc)
    dataset_size = find_dataset_size(html_doc)
    dataset_viewer = find_dataset_viewer2(html_doc)
    print(page_link,download_time,dataset_numb,dataset_size,dataset_viewer )


In [12]:
# 查询数据详细信息
# data.csv
bdirs = os.listdir(html_path)
data_path = os.path.join(viewer_path,'data2.csv')
f = open(data_path,'w',encoding='utf-8')
fieldnames = ['page_link','download_time','dataset_size','dataset_numb','bclass','sclass','dataset_viewer']
writer = csv.DictWriter(f,fieldnames=fieldnames)
writer.writeheader()
for bdir in tqdm(bdirs):
    # 打开大类文件夹
    # print(bdir)
    bdirs = os.path.join(html_path,bdir)
    # 大类
    print(bdir) 
    sdirs = os.listdir(bdirs)
    # # print(sdirs)
    for sdir in sdirs:
    #     # 逐个搜索小类文件夹,sdirs为小类集合
        # 小类
        print(' ',sdir)
        sdir_path = os.path.join(bdirs,sdir,'doc')
        sdocs = os.listdir(sdir_path)
        for sdoc in sdocs:
            # 数据名
            # print('  ',sdoc)
            sdoc_path = os.path.join(sdir_path,sdoc)
 
            with open(sdoc_path,'br') as f:
                text = f.read()
            try:
                text = str(text)
                info_dic = aio(text)
                info_dic['bclass'] = bdir
                info_dic['sclass'] = sdir
                writer.writerow(info_dic)
            except Exception as e:
                print('ERRORe',e,sdoc_path)
                pass
f.close()

In [20]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import torch
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
from torch.utils import data
from torch.optim import lr_scheduler
import collections
import re
import ast
import requests
from bs4 import BeautifulSoup as bs
from sentence_transformers import SentenceTransformer,CrossEncoder
from transformers import AutoModel
import matplotlib.pyplot as plt


if os.path.exists(r'Hugging _dataset.html'):
    with open(r'Hugging _dataset.html','r',encoding='utf-8') as f:
        dataset_html = f.read()
else:
    dataset_web_respond = requests.get(url=r'https://huggingface.co/datasets',verify=False)
    dataset_html = dataset_web_respond.text
soup = bs(dataset_html,'html.parser')
tags = soup.select('div.mb-20')[0].contents
btags = []
for i in tags:
    try:
        if 'mb-3' in i.get('class'):
            btags.append(i)
    except Exception:
        pass
cls = dict()
scls = []
for btag in btags:
    blabel = btag.div.string.rstrip('\t').rstrip('\n')
    temp = []
    for a in btag.find_all('a'):
        slabel = a.span.string
        scls.append(slabel)
        temp.append(slabel)
    cls[blabel]=temp
s_class_count = 0
for k,v in cls.items():
    s_class_count += len(v)
    print(k,v)
print('小类总数',s_class_count)


c2h = dict()
label = np.eye(46,46,dtype=np.float64)
k = 0
for _,v in cls.items():
    for c in v :
      c2h[c.replace(' ','_')] = label[k]
      k += 1
      

df = pd.read_csv(r'viewer\data2.csv')

df = df.loc[:,
     ['dataset_viewer',
      'page_link',
      'download_time',
      'dataset_size',
      'dataset_numb',
      'bclass',
      'sclass']
     ]


     
df.iloc[:,0] = df.iloc[:,0].apply(lambda x:x.replace('\\n','').replace('\\t',''))
df.iloc[:,1] = df.iloc[:,1].apply(lambda x:x.replace('https://huggingface.co/datasets/',''))
df.fillna(value=-1,inplace=True)


def generate_embeddings():
        """
        文本转向量
        """
        # 返回numpy数据
        TextEmbedding_model = AutoModel.from_pretrained(r'jina_embeddings_v2_small_en', trust_remote_code=True)
        return TextEmbedding_model
embedding_model = generate_embeddings()

def m_embedding(text):
    embeddings = embedding_model.encode(text,max_length=3487)
    return embeddings

x_encode = df.iloc[:,0:2].applymap(lambda x:m_embedding(x) if type(x)==str else x)
x = x_encode.to_numpy()
x_ = []
for i in range(x.shape[0]):
    ls = []
    x_.append(list(np.append(x[i][0],x[i][1])))
x = np.array(x_)
x1 = x


x_ = df.iloc[:,2:5]
def f(x):
    try:
        x = float(x)
    except Exception:
        x = -1
    return x
    
x2 = x_.applymap(f).fillna(value=-1).to_numpy()
x = np.concatenate((x2,x1), axis=1)

y = df.iloc[:,-1].apply(lambda x:np.array(c2h[x]))
y = np.vstack(y)

np.save('x',x)
np.save('y',y)











Multimodal ['Visual Question Answering']
Computer Vision ['Depth Estimation', 'Image Classification', 'Object Detection', 'Image Segmentation', 'Text-to-Image', 'Image-to-Text', 'Image-to-Image', 'Image-to-Video', 'Unconditional Image Generation', 'Video Classification', 'Text-to-Video', 'Zero-Shot Image Classification', 'Mask Generation', 'Zero-Shot Object Detection', 'Text-to-3D', 'Image-to-3D', 'Image Feature Extraction']
Natural Language Processing ['Text Classification', 'Token Classification', 'Table Question Answering', 'Question Answering', 'Zero-Shot Classification', 'Translation', 'Summarization', 'Feature Extraction', 'Text Generation', 'Text2Text Generation', 'Fill-Mask', 'Sentence Similarity', 'Table to Text', 'Multiple Choice', 'Text Retrieval']
Audio ['Text-to-Speech', 'Text-to-Audio', 'Automatic Speech Recognition', 'Audio-to-Audio', 'Audio Classification', 'Voice Activity Detection']
Tabular ['Tabular Classification', 'Tabular Regression', 'Tabular to Text', 'Time Seri

KeyboardInterrupt: 

In [56]:
x,y = torch.load('data.data')
x = pd.DataFrame(x)
y = pd.DataFrame(y)
print(x.shape,y.shape)
n = pd.concat([x,y],axis=1)
n.shape
print(n.iloc[:,:1030].duplicated().sum())
print(n.shape)

non_duplicate_indices = n.iloc[:, :1030].drop_duplicates().index
n = n.loc[non_duplicate_indices]
print(n.shape)
x = torch.Tensor(n.iloc[:,:1030].values)
y = torch.Tensor(n.iloc[:,1030:].values)
print(x.shape,y.shape)

(21955, 1030) (21955, 46)
6608
(21955, 1076)
(15347, 1076)
torch.Size([15347, 1030]) torch.Size([15347, 46])


AttributeError: expected 'f' to be string, path, or a file-like object with a 'write' attribute

In [58]:
torch.save([x,y],'data')


In [104]:
x.to_numpy()

In [105]:
x.shape

(15347, 1024)

In [71]:
from sentence_transformers import SentenceTransformer, util
cosine_fn = util.cos_sim

df = pd.DataFrame(np.zeros([15347, 2*46]))
df

,0,1
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
15342,0.0,0.0
15343,0.0,0.0
15344,0.0,0.0
15345,0.0,0.0


['Visual Question Answering']

In [89]:
list(s2e.keys())

['Visual Question Answering',
 'Depth Estimation',
 'Image Classification',
 'Object Detection',
 'Image Segmentation',
 'Text-to-Image',
 'Image-to-Text',
 'Image-to-Image',
 'Image-to-Video',
 'Unconditional Image Generation',
 'Video Classification',
 'Text-to-Video',
 'Zero-Shot Image Classification',
 'Mask Generation',
 'Zero-Shot Object Detection',
 'Text-to-3D',
 'Image-to-3D',
 'Image Feature Extraction',
 'Text Classification',
 'Token Classification',
 'Table Question Answering',
 'Question Answering',
 'Zero-Shot Classification',
 'Translation',
 'Summarization',
 'Feature Extraction',
 'Text Generation',
 'Text2Text Generation',
 'Fill-Mask',
 'Sentence Similarity',
 'Table to Text',
 'Multiple Choice',
 'Text Retrieval',
 'Text-to-Speech',
 'Text-to-Audio',
 'Automatic Speech Recognition',
 'Audio-to-Audio',
 'Audio Classification',
 'Voice Activity Detection',
 'Tabular Classification',
 'Tabular Regression',
 'Tabular to Text',
 'Time Series Forecasting',
 'Reinforcemen

In [122]:
type(df.iloc[i,0])
cosine_fn(v,x[i,:512]).numpy()[0,0]

0.68405753

In [126]:
s2e

{'Visual Question Answering': array([-4.27760363e-01, -5.70828974e-01, -6.14773892e-02,  1.11954665e+00,
        -5.38973697e-03, -2.98264533e-01, -6.44860193e-02,  2.31843576e-01,
         2.16084421e-01,  6.49074614e-02,  6.42325222e-01,  2.49732018e-01,
        -5.09906948e-01, -1.40466273e-01,  4.02813077e-01, -6.05001211e-01,
        -7.66328275e-02, -3.44820805e-02, -2.30921373e-01, -3.88772696e-01,
        -4.54473376e-01, -3.83605897e-01,  3.39414060e-01, -6.88306570e-01,
         6.36716187e-01, -8.12577069e-01, -4.40945357e-01, -1.79314941e-01,
         2.52624929e-01,  3.27398062e-01, -3.82801086e-01,  4.19463627e-02,
        -5.64870417e-01,  9.53722186e-03, -4.13520813e-01,  1.11914091e-01,
         2.60914415e-01, -8.27111006e-02, -2.17007503e-01, -2.94038415e-01,
        -2.70839572e-01,  1.38497174e-01,  6.12399757e-01,  7.82192349e-01,
        -4.70167994e-01, -2.90438440e-02, -6.47838235e-01,  5.98980665e-01,
         3.77118766e-01, -5.54123580e-01, -4.19284225e-01, 

In [128]:
s2e = dict()
for k,v in cls.items():
    s_class_count += len(v)
    for s in v:
        lab = m_embedding(k+s)
        s2e[s] = lab

for i in range(15347):
    for j,(k,v) in enumerate(s2e.items()):
        df.iloc[i,j] = cosine_fn(v,x[i,:512]).numpy()[0,0]
        df.iloc[i,j+46] = cosine_fn(v,x[i,512:]).numpy()[0,0]



In [139]:
x = pd.DataFrame(x)

In [143]:
df = pd.concat([x.iloc[:,:6],df],axis=1).shape

In [144]:
my_array = np.array(df)
my_tensor = torch.tensor(my_array)
torch.save(my_tensor,'data')

In [26]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("all-MiniLM-L6-v2")

# Single list of sentences
sentences = [
    "The cat sits outside",
    "A man is playing guitar",
    "I love pasta",
    "The new movie is awesome",
    "The cat plays in the garden",
    "A woman watches TV",
    "The new movie is so great",
    "Do you like pizza?",
]

# Compute embeddings
embeddings = model.encode(sentences, convert_to_tensor=True)

# Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.cos_sim(embeddings, embeddings)

# Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(cosine_scores.shape[0]):
    for j in range(cosine_scores.shape[1]):
        pairs.append({"index": [i, j], "score": cosine_scores[i][j]})

# Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x["score"], reverse=True)

for pair in pairs[0:10]:
    i, j = pair["index"]
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        sentences[i], sentences[j], pair["score"]
    ))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\elysia\.conda\envs\v1\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\elysia\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

The cat sits outside 		 The cat sits outside 		 Score: 1.0000
A man is playing guitar 		 A man is playing guitar 		 Score: 1.0000
I love pasta 		 I love pasta 		 Score: 1.0000
The new movie is awesome 		 The new movie is awesome 		 Score: 1.0000
The new movie is so great 		 The new movie is so great 		 Score: 1.0000
Do you like pizza? 		 Do you like pizza? 		 Score: 1.0000
The cat plays in the garden 		 The cat plays in the garden 		 Score: 1.0000
A woman watches TV 		 A woman watches TV 		 Score: 1.0000
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
The new movie is so great 		 The new movie is awesome 		 Score: 0.8939
